# Indexación y selección de datos

## Selección de datos en Series

Como vimos anteriormente, un objeto ``Series`` actúa en muchos aspectos como un array unidimensional de NumPy, y en muchos aspectos como un diccionario estándar de Python.
Si mantenemos en mente estas dos analogías superpuestas, nos ayudará a entender los patrones de indexación y selección de datos en estos arrays.

### Series como diccionario

Al igual que un diccionario, el objeto ``Series`` proporciona una correspondencia entre una colección de claves y una colección de valores:

In [ ]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

In [ ]:
data['b']

También podemos utilizar expresiones y métodos de Python tipo diccionario para examinar las claves/índices y los valores:

In [ ]:
'a' in data

In [ ]:
data.keys()

In [ ]:
list(data.items())

Los objetos ``Series`` pueden incluso modificarse con una sintaxis similar a la de un diccionario.
Del mismo modo que se puede ampliar un diccionario asignando una nueva clave, se puede ampliar una ``Series`` asignando un nuevo valor de índice:

In [ ]:
data['e'] = 1.25
data

Esta fácil mutabilidad de los objetos es una característica conveniente: bajo el capó, Pandas está tomando decisiones sobre la disposición de la memoria y la copia de datos que pueda ser necesario realizar; el usuario generalmente no necesita preocuparse por estos temas.

### Series como array unidimensional

Una ``Series`` se basa en esta interfaz tipo diccionario y proporciona una selección de elementos tipo array a través de los mismos mecanismos básicos que los arrays de NumPy, es decir, *slicing*, *enmascaramiento* e *índice fancy*.
Algunos ejemplos son los siguientes:

In [ ]:
# slicing por índice explícito
data['a':'c']

In [ ]:
# slicing por índice entero implícito
data[0:2]

In [ ]:
# enmascaramiento
data[(data > 0.3) & (data < 0.8)]

In [ ]:
# indexación fancy 
data[['a', 'e']]

Entre ellos, el slicing puede ser la fuente de mayor confusión.
Fíjese en que cuando se trocea con un índice explícito (es decir, ``data['a':'c']``), el índice final se *incluye* en el slicing, mientras que cuando se trocea con un índice implícito (es decir, ``data[0:2]``), el índice final se *excluye*.

### Indexadores: loc e iloc

Estas convenciones de slicing e indexación pueden ser fuente de confusión.
Por ejemplo, si tu ``Series`` tiene un índice entero explícito, una operación de indexación como ``datos[1]`` utilizará los índices explícitos, mientras que una operación de corte como ``datos[1:3]`` utilizará el índice implícito estilo Python.

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

In [ ]:
# índice explícito al indexar
data[1]

In [ ]:
# índice implícito al hacer un slicing
data[1:3]

Debido a esta confusión potencial en el caso de índices enteros, Pandas proporciona algunos atributos *indexer* especiales que exponen explícitamente ciertos esquemas de indexación.
No se trata de métodos funcionales, sino de atributos que exponen una interfaz de slicing particular para los datos de la ``Series``.

En primer lugar, el atributo ``loc`` permite la indexación y el corte que siempre hace referencia al índice explícito:

In [ ]:
data.loc[1]

In [ ]:
data.loc[1:3]

El atributo ``iloc`` permite la indexación y la segmentación que siempre hace referencia al índice implícito de estilo Python:

In [ ]:
data.iloc[1]

In [ ]:
data.iloc[1:3]

## Selección de datos en DataFrame

Recordemos que un ``DataFrame`` actúa en muchos aspectos como un array bidimensional o estructurado, y en otros como un diccionario de estructuras ``Series`` que comparten el mismo índice.
Estas analogías pueden ser útiles a la hora de explorar la selección de datos dentro de esta estructura.

### DataFrame como diccionario

La primera analogía que vamos a considerar es el ``DataFrame`` como diccionario de objetos ``Series`` relacionados.
Volvamos a nuestro ejemplo de las áreas y poblaciones de los estados:

In [ ]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

Se puede acceder a las ``Series`` individuales que componen las columnas del ``DataFrame`` mediante la indexación de tipo diccionario del nombre de la columna:

In [ ]:
data['area']

De forma equivalente, podemos utilizar el acceso de tipo atributo con nombres de columna que sean cadenas:

In [ ]:
data.area

Este acceso de columna de tipo atributo accede en realidad exactamente al mismo objeto que el acceso de tipo diccionario:

In [ ]:
data.area is data['area']

Aunque se trata de una abreviatura útil, hay que tener en cuenta que no funciona en todos los casos.
Por ejemplo, si los nombres de las columnas no son cadenas, o si los nombres de las columnas entran en conflicto con métodos del ``DataFrame``, este acceso tipo atributo no es posible.
Por ejemplo, el ``DataFrame`` tiene un método ``pop()``, por lo que ``data.pop`` apuntará a éste en lugar de a la columna ``"pop"``:

In [ ]:
data.pop is data['pop']

En particular, debe evitar la tentación de intentar asignar columnas mediante atributos (es decir, utilice ``datos['pop'] = z`` en lugar de ``datos.pop = z``).

Al igual que con los objetos ``Series`` comentados anteriormente, esta sintaxis de tipo diccionario también se puede utilizar para modificar el objeto, en este caso añadiendo una nueva columna:

In [ ]:
data['density'] = data['pop'] / data['area']
data

### DataFrame como matriz bidimensional

Como se ha mencionado anteriormente, también podemos ver el ``DataFrame`` como una matriz bidimensional mejorada.
Podemos examinar la matriz de datos subyacente utilizando el atributo ``values``:

In [ ]:
data.values

Con esta imagen en mente, se pueden hacer muchas observaciones familiares sobre el propio ``DataFrame``.
Por ejemplo, podemos transponer el ``DataFrame`` completo para intercambiar filas y columnas:

In [ ]:
data.T

Sin embargo, cuando se trata de indexar objetos ``DataFrame``, está claro que el estilo de indexación de diccionario de las columnas excluye nuestra capacidad de tratarlo simplemente como un array NumPy.
En particular, al pasar un único índice a un array se accede a una fila:

In [ ]:
data.values[0]

y pasando un único "índice" a un ``DataFrame`` se accede a una columna:

In [ ]:
data['area']

Por lo tanto, para la indexación estilo array, necesitamos otra convención.
Aquí Pandas utiliza de nuevo los indexadores ``loc``e ``iloc`` mencionados anteriormente.
Usando el indexador ``iloc``, podemos indexar el array subyacente como si fuera un simple array NumPy (usando el índice implícito estilo Python), pero el índice ``DataFrame`` y las etiquetas de las columnas se mantienen en el resultado:

In [ ]:
data.iloc[:3, :2]

Del mismo modo, utilizando el indexador ``loc`` podemos indexar los datos subyacentes en un estilo similar a un array pero utilizando el índice explícito y los nombres de las columnas:

In [ ]:
data.loc[:'Illinois', :'pop']

Dentro de estos indexadores se puede utilizar cualquiera de los patrones de acceso a datos conocidos del estilo NumPy.
Por ejemplo, en el indexador ``loc`` podemos combinar enmascaramiento e indexación fancy como en lo siguiente:

In [ ]:
data.loc[data.density > 100, ['pop', 'density']]

Cualquiera de estas convenciones de indexación también se puede utilizar para establecer o modificar valores; esto se hace de la manera estándar a la que puede estar acostumbrado de trabajar con NumPy:

In [ ]:
data.iloc[0, 2] = 90
data

### Convenciones de indexación adicionales

Hay un par de convenciones de indexación adicionales que pueden parecer contradictorias con lo expuesto anteriormente, pero que sin embargo pueden ser muy útiles en la práctica.
En primer lugar, mientras que *indexar* se refiere a columnas, *slicing* se refiere a filas:

In [ ]:
data['Florida':'Illinois']

Estos slicing también pueden referirse a filas por número en lugar de por índice:

In [ ]:
data[1:3]

Del mismo modo, las operaciones de enmascaramiento directo también se interpretan por filas en lugar de por columnas:

In [ ]:
data[data.density > 100]

Estas dos convenciones son sintácticamente similares a las de un array de NumPy, y aunque no se ajusten exactamente al molde de las convenciones de Pandas, son bastante útiles en la práctica.

<!--NAVIGATION-->
< [Introducción a Objectos Pandas](1-Objetos_Pandas.ipynb) | [Operar con datos en Pandas](3-Operaciones_en_Pandas.ipynb) >